In [2]:
import csv
import math

def readDataFromCSV(filename):
    dict_list = []
    reader = csv.DictReader(open(filename, 'rb'))
    for line in reader:
        dict_list.append(line)
    return dict_list

# Write a "Feature - Category - Value" type dictionary to a CSV
def writeEpsilonsDataToCSV(filename, epsdict):
    with open(filename, 'w') as file:
        for feature in epsdict.keys():
            for category in epsdict[feature].keys():
                epsilon = epsdict[feature][category]["epsilon"]
                nx = epsdict[feature][category]["nx"]
                nc = epsdict[feature][category]["nc"]
                ncx = epsdict[feature][category]["ncx"]
                file.write("%s,%s,%s,%s,%s,%s\n"%(feature, category, epsilon, nx, nc, ncx))

def writeDictDataToCSV(filename, dict):
    csv_columns = getAllFeatures(dict)
    with open(filename, 'w') as file:
        writer = csv.DictWriter(file, fieldnames = csv_columns)
        writer.writeheader()
        for row in dict:
            writer.writerow(row)

def writeCompleteDictDataToCSV(filename, dict):
    csv_columns = getAllFeatures(dict)
    csv_columns.append(cClass)
    with open(filename, 'w') as file:
        writer = csv.DictWriter(file, fieldnames = csv_columns)
        writer.writeheader()
        for row in dict:
            writer.writerow(row)

def getAllFeatures(data):
    features = []
    for feature in data[0]:
        if feature != cClass:
            features.append(feature)
    return features

def getFeatures(data):
    features = []
    for feature in data[0]:
        if feature != 'FID' and feature != cClass:
            features.append(feature)
    return features

def getCategories(feature, data):
    mincount = int(data[0][feature])
    maxcount = int(data[0][feature])
    for id in range(len(data)):
        current = int(data[id][feature])
        if current > maxcount:
            maxcount = current
        if current < mincount:
            mincount = current
    return mincount, maxcount

def getCategoriesT(feature, data):
    cat_dict = {}
    print 'feature: ' + feature
    for i in range(len(data)):
        current = str(data[i][feature])
        cat_dict[current] = "cat"
    return cat_dict.keys()

def getNX(feature, category, data):
    count = 0
    for i in range(len(data)):
        if str(category) == str(data[i][feature]):
            count = count + 1
    #fc = str(feature) + ' (Cat: ' + str(category) + ')'
    #print fc + ': ' + str(count)
    return count

def getNCX(feature1, category1, feature2, category2, data):
    count = 0
    for i in range(len(data)):
        if str(category1) == str(data[i][feature1]) and str(category2) == str(data[id][feature2]):
            count = count + 1
    #fc1 = str(feature1) + ' (Cat: ' + str(category1) + ')'
    #fc2 = str(feature2) + ' (Cat: ' + str(category2) + ')'
    #print fc1 + ' AND ' + fc2 + ': ' + str(count)
    return count

def getAllEpsilons(data):
    features = getFeatures(data)
    epsilons = {}
    for feature in features:
        epsilons[feature] = getEpsilonsFromFeature(feature, data)
    return epsilons

def getEpsilonsFromFeature(feature, data):
    categories = getCategoriesT(feature, data)
    epsilons = {}
    for category in categories:
        values = getEpsilon(feature, category, data)
        epsilons[category] = values
    return epsilons

def getEpsilon(feature, category, data):
    n = len(data)
    nx = getNX(feature, category, data)
    nc = getNX(cClass, 1, data)
    ncx = getNCX(feature, category, cClass, cCategory, data)
    if n != 0 and nx != 0 and nc != 0:
        pc = nc / float(n)
        pcx = ncx / float(nx)
        epsilon = nx * (pcx - pc) / math.sqrt(nx * pc * (1 - pc))
    else:
        epsilon = 0
        
    values = {"epsilon": epsilon, "nx": nx, "nc": nc, "ncx": ncx}
    #print 'Epsilon :' + str(epsilon)
    return values

# Get a smoothed epsilon by using a Laplace Estimator with alpha = 1, beta = |C|
def getSmoothedEpsilon(feature, category, data, beta):
    n = len(data)
    nx = getNX(feature, category, data) + beta
    nc = getNX(cClass, cCategory, data)
    ncx = getNCX(feature, category, cClass, cCategory, data) + 1
    pc = nc / float(n)
    pcx = ncx / float(nx)
    epsilon = nx * (pcx - pc) / math.sqrt(nx * pc * (1 - pc))
    return epsilon

#training_data = readDataFromCSV('datos_completos.csv')
#test_data = readDataFromCSV('datos_completos.csv')
#all_data = readDataFromCSV('datos_completos.csv')

#fl = ["PPERSAUT", "PBRAND", "APLEZIER", "MKOOPKLA", "MOSTYPE", "MOSHOOFD",
#"APERSAUT", "PWAPART", "AWAPART", "ABYSTAND", "MHHUUR", "MHKOOP", "PPLEZIER"]
#scores_nsm = getScoresFromFeatures(fl, training_data, training_data, all_data, 0)
#scores_sm = getScoresFromFeatures(fl, training_data, training_data, all_data, 1)
#writeCompleteDictDataToCSV('scores_trainingset_15F_nsm.csv', scores_nsm)
#writeCompleteDictDataToCSV('scores_trainingset_15F_sm.csv', scores_sm)
#scores_nsm = getScoresFromFeatures(fl, training_data, test_data, all_data, 0)
#scores_sm = getScoresFromFeatures(fl, training_data, test_data, all_data, 1)
#writeCompleteDictDataToCSV('scores_testset_15F_nsm.csv', scores_nsm)
#writeCompleteDictDataToCSV('scores_testset_15F_sm.csv', scores_sm)

In [3]:
cClass = 'Obesidad'
cCategory = 1
datafile = 'snpsobes.csv'

training_data = readDataFromCSV(datafile)
test_data = readDataFromCSV(datafile)
all_data = readDataFromCSV(datafile)

eps = getAllEpsilons(datafile)

feature: s


TypeError: string indices must be integers, not str